In [1]:
!pip install -q streamlit
!pip install pytorch-lightning==1.5.10
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipyth

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.594s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [2]:
%%writefile app.py
import streamlit as st

import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision.datasets import FashionMNIST

import matplotlib.pyplot as plt
import pytorch_lightning as pl

import numpy as np

from PIL import Image

import zipfile

# random_seed = 42
# torch.manual_seed(random_seed)
BATCH_SIZE = 128
AVAIL_GPUS = min(1, torch.cuda.device_count())
NUM_WORKERS = int(os.cpu_count() / 2)

class FMNISTDataModule(pl.LightningDataModule):
  def __init__ (self, data_dir='./data', batch_size = BATCH_SIZE, num_workers=NUM_WORKERS):
    """
    Initializes the FMNISTDataModule.

    Args:
        data_dir (str): Directory to store the dataset.
        batch_size (int): Size of the mini-batches during training.
        num_workers (int): Number of workers for DataLoader.
    """
    super().__init__()
    self.data_dir = data_dir
    self.batch_size = batch_size
    self.num_workers = num_workers

    # Define the data transformation pipeline
    self.transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,)),
        ]
    )

  def prepare_data(self):
    """
    Downloads and prepares the FashionMNIST dataset.
    """
    FashionMNIST(self.data_dir, train=True, download=True)
    FashionMNIST(self.data_dir, train=False, download=True)

  def setup(self, stage=None):
    """
    Splits the dataset into training, validation, and test sets.

    Args:
        stage (str, optional): 'fit' for training/validation, 'test' for testing.
    """
    if stage=="fit" or stage is None:
      fmnist_full = FashionMNIST(self.data_dir, train=True, transform=self.transform)
      self.fmnist_train, self.fmnist_val = random_split(fmnist_full, [55000, 5000])

    if stage=="test" or stage is None:
      self.fmnist_test = FashionMNIST(self.data_dir, train=False, transform=self.transform)

  def train_dataloader(self):
    """
    Returns DataLoader for the training set.
    """
    return DataLoader(self.fmnist_train, batch_size=self.batch_size, num_workers=self.num_workers)

  def val_dataloader(self):
    """
    Returns DataLoader for the validation set.
    """
    return DataLoader(self.fmnist_val, batch_size=self.batch_size, num_workers=self.num_workers)

  def test_dataloader(self):
    """
    Returns DataLoader for the test set.
    """
    return DataLoader(self.fmnist_test, batch_size=self.batch_size, num_workers=self.num_workers)


class MNISTDataModule(pl.LightningDataModule):
  def __init__ (self, data_dir='./data', batch_size = BATCH_SIZE, num_workers=NUM_WORKERS):
    """
    Initializes the MNISTDataModule.

    Args:
        data_dir (str): Directory to store the dataset.
        batch_size (int): Size of the mini-batches during training.
        num_workers (int): Number of workers for DataLoader.
    """
    super().__init__()
    self.data_dir = data_dir
    self.batch_size = batch_size
    self.num_workers = num_workers

    self.transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,)),
        ]
    )

  def prepare_data(self):
    """
    Downloads and prepares the MNIST dataset.
    """
    MNIST(self.data_dir, train=True, download=False)
    MNIST(self.data_dir, train=False, download=True)

  def setup(self, stage=None):
    """
    Splits the dataset into training, validation, and test sets.

    Args:
        stage (str, optional): 'fit' for training/validation, 'test' for testing.
    """
    if stage=="fit" or stage is None:
      mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
      self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

    if stage=="test" or stage is None:
      self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

  def train_dataloader(self):
    """
    Returns DataLoader for the training set.
    """
    return DataLoader(self.mnist_train, batch_size=self.batch_size, num_workers=self.num_workers)

  def val_dataloader(self):
    """
    Returns DataLoader for the validation set.
    """
    return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=self.num_workers)

  def test_dataloader(self):
    """
    Returns DataLoader for the test set.
    """
    return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=self.num_workers)

# Define the Discriminator neural network class
class Discriminator(nn.Module):
  def __init__(self):
    """
    Initializes the Discriminator neural network.

    The architecture consists of convolutional layers, max-pooling, dropout, and fully connected layers.

    Input: 1 channel (grayscale image)
    Output: 1 (sigmoid activation for binary classification)
    """
    super().__init__()

    # First convolutional layer with 5x5 kernel, input channels=1, output channels=10
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)

    # Second convolutional layer with 5x5 kernel, input channels=10, output channels=20
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)

    # Dropout layer to prevent overfitting
    self.conv2d_drop = nn.Dropout2d()

    # Fully connected layers
    self.fc1 = nn.Linear(320, 50)     # 320 input features, 50 output features
    self.fc2 = nn.Linear(50, 1)       # 50 input features, 1 output feature

  def forward(self, x):
    """
    Defines the forward pass of the Discriminator.

    Args:
        x (torch.Tensor): Input tensor.

    Returns:
        torch.Tensor: Output tensor with sigmoid activation for binary classification.
    """

    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2d_drop(self.conv2(x)), 2))

    # Reshape the tensor for the fully connected layers
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)

    # Final fully connected layer with sigmoid activation for binary classification
    x = self.fc2(x)
    return torch.sigmoid(x)

# Define the Generator neural network class
class Generator(nn.Module):
  def __init__(self, latent_dim):
    """
    Initializes the Generator neural network.

    The architecture consists of linear and transpose convolutional layers to generate images.

    Args:
        latent_dim (int): Dimensionality of the latent space.
    """
    super().__init__()

    # Linear layer to transform the latent space to the initial convolutional shape
    self.lin1 = nn.Linear(latent_dim, 7*7*64)

    # Transpose convolutional layers to upsample the data
    self.ct1 = nn.ConvTranspose2d(64, 32, 4, stride=2)
    self.ct2 = nn.ConvTranspose2d(32, 16, 4, stride=2)

    # Final convolutional layer to generate the output image
    self.conv = nn.Conv2d(16, 1, kernel_size=7)

  def forward(self, x):
    """
    Defines the forward pass of the Generator.

    Args:
        x (torch.Tensor): Input tensor from the latent space.

    Returns:
        torch.Tensor: Output tensor representing the generated image.
    """
    # Linear layer to transform the input from latent space to initial convolutional shape
    x = self.lin1(x)
    x = F.relu(x)

    # Reshape the tensor for the transpose convolutional layers
    x = x.view(-1, 64, 7, 7)

    x = self.ct1(x)
    x = F.relu(x)

    x = self.ct2(x)
    x = F.relu(x)

    # Final convolutional layer to generate the output image
    return self.conv(x)


# Define the GAN LightningModule class
class GAN(pl.LightningModule):

  def __init__(self, latent_dim=100, lr=0.0002):
    """
    Initializes the GAN LightningModule.

    Args:
        latent_dim (int): Dimensionality of the latent space.
        lr (float): Learning rate for optimization.
    """
    super().__init__()
    self.save_hyperparameters()

    self.generator = Generator(latent_dim=self.hparams.latent_dim)
    self.discriminator = Discriminator()

    # Pre-generate random noise for validation
    self.validation_z = torch.randn(6, self.hparams.latent_dim)

  def forward(self, z):
    """
    Defines the forward pass of the GAN.

    Args:
        z (torch.Tensor): Input tensor from the latent space.

    Returns:
        torch.Tensor: Output tensor representing the generated image.
    """
    return self.generator(z)

  def adversarial_loss(self, y_hat, y):
    """
    Computes the adversarial loss using binary cross-entropy.

    Args:
        y_hat (torch.Tensor): Predictions.
        y (torch.Tensor): Ground truth labels.

    Returns:
        torch.Tensor: Adversarial loss.
    """
    return F.binary_cross_entropy(y_hat, y)

  def training_step(self, batch, batch_idx, optimizer_idx):
    """
    Defines the training step for both the Generator and the Discriminator.

    Args:
        batch: Batch of real images.
        batch_idx: Index of the current batch.
        optimizer_idx: Index of the optimizer being used (0 for Generator, 1 for Discriminator).

    Returns:
        dict: Dictionary containing the loss and logging information.
    """

    # Getting real images from the batch
    real_imgs, _ = batch

    # Creating random noise
    z = torch.randn(real_imgs.shape[0], self.hparams.latent_dim)
    z = z.type_as(real_imgs)

    # For Generator
    if optimizer_idx == 0:

      # The forward step - which creates images using generator from random noise
      fake_imgs = self(z)

      # The output from the discriminator
      # 1 means the discriminator predicted this as a real image, meaning the generator did a great job
      # 0 means the discriminator predicted this as a fake image. The generator needs to improve
      y_hat = self.discriminator(fake_imgs)
      y = torch.ones(real_imgs.size(0), 1)

      y = y.type_as(real_imgs)

      g_loss = self.adversarial_loss(y_hat, y)

      log_dict = {"g_loss": g_loss}

      return {"loss": g_loss, "progress_bar": log_dict, "log": log_dict}

    # For Discriminator
    if optimizer_idx == 1:
      y_hat_real = self.discriminator(real_imgs)

      y_real = torch.ones(real_imgs.size(0), 1)
      y_real = y_real.type_as(real_imgs)

      real_loss = self.adversarial_loss(y_hat_real, y_real)

      y_hat_fake = self.discriminator(self(z).detach())
      y_fake = torch.zeros(real_imgs.size(0), 1)
      y_fake = y_fake.type_as(real_imgs)

      fake_loss = self.adversarial_loss(y_hat_fake, y_fake)

      d_loss = (real_loss + fake_loss) / 2
      log_dict = {"d_loss": d_loss}

      return {"loss": d_loss, "progress_bar": log_dict, "log": log_dict}


  def configure_optimizers(self):
    """
    Configures the optimizers for the Generator and Discriminator.

    Returns:
        list: List of optimizers.
    """
    lr = self.hparams.lr
    opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr)
    opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr)
    return [opt_g, opt_d], []

  def plot_imgs(self):
    """
    Plots generated images using the current Generator state.
    """
    z = self.validation_z.type_as(self.generator.lin1.weight)
    sample_imgs = self(z).cpu()

    print('epoch ', self.current_epoch)
    fig = plt.figure()

    for i in range(sample_imgs.size(0)):
      plt.subplot(2, 3, i + 1)
      plt.tight_layout()
      plt.imshow(sample_imgs.detach()[i, 0, :, :], cmap = 'gray_r', interpolation = 'none')
      plt.title("Generated Data")
      plt.xticks([])
      plt.yticks([])
      plt.axis('off')
    plt.show()

  def on_epoch_end(self):
    """
    Hook function called at the end of each epoch.
    """
    self.plot_imgs()


def download_img(ds_type, number):
  """
  Downloads and saves generated images from a trained GAN model.

  Args:
      ds_type (str): Dataset type ('Numbers' or 'Fashion').
      number (int): Number of images to generate and save.
  """
  TRANSFORM = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,)),
        ]
    )

  # This is just to tell the number of fake images to generate
  mnist_val = MNIST("./data", download=True, train=False, transform=TRANSFORM)
  for batch in DataLoader(mnist_val, batch_size=number, num_workers=NUM_WORKERS):
    img, _ = batch
    break

  # Generate random noise for the latent space
  z = torch.randn(img.shape[0], 100)
  z = z.type_as(img)

  # Load the GAN model based on the dataset type
  if ds_type == 'Numbers':
    model2 = GAN.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/Saved_Models/GAN/GAN1.ckpt")

  elif ds_type == 'Fashion':
    model2 = GAN.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/Saved_Models/GAN/GAN2.ckpt")

  else:
    return

  # Generate fake images using the loaded GAN model
  fake_imgs = model2(z)

  if os.path.exists('/content/For_Download') == False:
    os.mkdir('/content/For_Download')

  # Save each generated image and display a random one
  rand_int = np.random.randint(0, number)
  for i in range(fake_imgs.size(0)):
    fig = plt.figure()
    plt.imshow(fake_imgs.detach()[i, 0, :, :], interpolation = 'none')
    plt.savefig(f"/content/For_Download/Img{i}.png")
    if i == rand_int:
      st.image(f'/content/For_Download/Img{i}.png', caption='Here is a sample image')

  # Create a ZipFile object for the folder that you want to zip
  zip_file = zipfile.ZipFile('/content/For_Download.zip', 'w')

  # Add all of the files in the folder to the zip file
  for file in os.listdir('/content/For_Download'):
      zip_file.write(os.path.join('/content/For_Download', file))

  # Close the zip file
  zip_file.close()

# Display a header with a rainbow divider
st.header('Training Data Augmentation using GANs', divider='rainbow')

# Create a form with a selectbox for dataset type and a slider for the number of images
with st.form('my_form'):
  ds_type = st.selectbox('For which dataset do you need more images?', ('None', 'Numbers', 'Fashion'))
  number = st.slider("Number of images you want", min_value=1, max_value=100, value=None)
  submitted = st.form_submit_button('Submit')
  st.write('Dataset you want more images for augmentation:', ds_type)
  st.write('Number of images you want:', number)
  download_img(ds_type, number)

if ds_type != "None":
  st.write("Your images are ready for download!")
  with open('/content/For_Download.zip', 'rb') as f:
    # Provide a download button for the generated zip file
    st.download_button('Download Zip', f, file_name='Images.zip')

Writing app.py


In [3]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.243.146.98
npx: installed 22 in 3.109s
your url is: https://rich-crabs-sin.loca.lt
